Victor CHAU & Tahar AMAIRI
# IFT 870 - Forage de données
## TP4 : Fonctions descriptives

### Sommaire

# TODO

## Importation des données

In [1]:
# import libs
from matplotlib import pyplot as plt
from IPython.display import display
import seaborn as sns
import pandas as pd
import numpy as np

# read CSV
data = pd.read_csv('TP4_data.csv')

# print the df
data.head()

,id,sequence
0,U10409.1/2025-2178,AACUUUCAGCAACGGAUCUCUUGGUUCUCGCAUCGAUGAAGAACGC...
1,AF223066.1/5881-6030,GACUCUCGGCAACGGAUAUCUCGGCUCUCGCAUCGAUGAAGAACGU...
2,Y07976.1/124-271,CACUUUCAGCAACGGAUGUCUCGGUUCCCAUAACGAUGAAGAACGC...
3,X03680.1/3159-3308,UAGCUUCAGCGAUGGAUCGGUUGCAUCGAGUAUCGAUGAAGAACGC...
4,X90410.1/15-165,AACUUUCAGCAAUGGAUCUCUUGGCUCUUGCAUCGAUGAAGAACGC...
